In [1]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import json
import ast
import re
from datetime import datetime
from zoneinfo import ZoneInfo

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)     

In [ ]:
# Load the dataset
df = pd.read_parquet('data/processed/final_joined.parquet')

In [ ]:
# remove duplicates

# Check original size
before_rows = len(df)

# Drop duplicates
df = df.drop_duplicates().reset_index(drop=True)

# Report result
after_rows = len(df)
removed = before_rows - after_rows

print(f"Removed {removed:,} duplicate rows. Final row count: {after_rows:,}")


In [ ]:
columns_to_check = [
    'title_review', 'text', 'images_review', 'description',
    'bought_together', 'subtitle', 'author', 'main_category',
    'asin', 'parent_asin', 'user_id', 'title_meta', 'features',
    'images_meta', 'store', 'categories'
]

# Track dropped or missing columns
dropped_cols = []

# Define helper to detect empty lists or their string form
def is_effectively_empty_list(val):
    return val == [] or val == '[]' or str(val).strip().lower() in {'[]', 'none', ''}

for col in columns_to_check:
    if col in df.columns:
        # Replace only effectively empty list-like values with NaN
        df[col] = df[col].apply(lambda x: np.nan if is_effectively_empty_list(x) else x)

        # Count nulls and calculate usable %
        null_count = df[col].isna().sum()
        total_rows = len(df)
        usable_pct = 100 * (1 - null_count / total_rows)

        print(f"{col} → Nulls: {null_count:,} / {total_rows:,} → Usable: {usable_pct:.2f}%")
    else:
        print(f"Column '{col}' no longer exists in the DataFrame.")
        dropped_cols.append(col)

# Summary of missing columns
if dropped_cols:
    print(f"\nSummary: The following columns were not found (reason: dropped – verify in the next cell): {', '.join(dropped_cols)}")
else:
    print("\nAll specified columns are present and processed.")


### Based on above result - Confirm columns to drop 

1. images_review
2. bought_together
3. subtitle
4. author

In addition: <br>
5. details <br>
6. average_rating #not applicable since its 1.5M of entire dataset only <br>
7. rating_number #not applicable since its 1.5M of entire dataset only <br>

In [ ]:
# Define columns to drop
processed_drop = ['images_review', 'bought_together', 'subtitle', 'author', 'details', 'average_rating', 'rating_number', 'videos']

# Drop only if they exist
existing_cols = [col for col in processed_drop if col in df.columns]
df.drop(columns=existing_cols, inplace=True)

# Confirm drop
print(f"Dropped columns: {existing_cols}")
print(f"Remaining columns: {df.columns.tolist()}")

# Show the updated DataFrame
df.head()

In [ ]:
# Columns setup
id_columns = {'asin', 'parent_asin', 'user_id'}
preserve_bracket_fields = {'description', 'features', 'images_meta', 'categories'}
text_lower_fields = {'title_review', 'text', 'title_meta', 'store'}

# Values to treat as null
null_like_values = {'none', '', ' ', '[]'}

# Cleaning function
def smart_clean(val, col):
    # ID columns: trim + set NaN if empty-like
    if col in id_columns:
        if isinstance(val, str):
            trimmed = val.strip()
            return np.nan if trimmed.lower() in null_like_values else trimmed
        return val

    # Bracket-sensitive: lowercase + NaN if empty-like
    if col in preserve_bracket_fields:
        if isinstance(val, str):
            raw = val.strip()
            return np.nan if raw.lower() in null_like_values else raw.lower()
        elif isinstance(val, list):
            return [str(item).strip().lower() for item in val if isinstance(item, str)]
        return val

    # Regular text fields
    if isinstance(val, str):
        val = val.strip().lower()
        return np.nan if val in null_like_values else val

    return val

# Apply per column
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].apply(lambda x: smart_clean(x, col))

print("Cleaned DataFrame: bracket fields preserved, lowercase applied, nulls handled.")

# Ensure numeric first, then convert to int
df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0).astype(int)

## Feature Engineering

In [ ]:
# Examine 'features' & 'description'
sample_feature = df['features'].dropna().iloc[0]
sample_description = df['description'].dropna().iloc[0]

print(sample_feature)
print(sample_description)

# Perform clean up for 'features' & 'description'
def clean_stringified_list(val):
    """
    Cleans stringified list fields like 'features' and 'description'.
    Returns a flattened, SBERT-friendly string.
    """
    if pd.isna(val):
        return ""
    try:
        parsed = ast.literal_eval(val)
        if isinstance(parsed, list):
            return " ".join([str(s).strip() for s in parsed])
    except:
        pass
    return str(val).strip()

# Apply separately to create two new clean columns
df['features_clean'] = df['features'].apply(clean_stringified_list)
df['description_clean'] = df['description'].apply(clean_stringified_list)

In [ ]:
# Examine 'images_meta'
sample_meta = df['images_meta'].dropna().iloc[0]
print(sample_meta)

'''
Transformed images_meta into 3 structured fields for downstream use:
- main_image_url (best image based on priority: hi_res > large > thumb)
- num_images (total image count)
- hi_res_images (list of hi-res or fallback large URLs)
'''

def extract_main_image_url(val):
    try:
        images = ast.literal_eval(val)
        if isinstance(images, list):
            for img in images:
                if img.get('variant') == 'main':
                    return img.get('hi_res') or img.get('large') or img.get('thumb')
            # fallback to first image
            img = images[0]
            return img.get('hi_res') or img.get('large') or img.get('thumb')
    except:
        return None

# Count total images
def count_images(val):
    try:
        images = ast.literal_eval(val)
        return len(images) if isinstance(images, list) else 0
    except:
        return 0

# Extract all hi_res (or fallback to large)
def extract_all_hi_res(val):
    try:
        images = ast.literal_eval(val)
        return [img.get('hi_res') or img.get('large') for img in images if isinstance(img, dict)]
    except:
        return []

# Apply all 3
df['main_image_url'] = df['images_meta'].apply(extract_main_image_url)
df['num_images'] = df['images_meta'].apply(count_images)
df['hi_res_images'] = df['images_meta'].apply(extract_all_hi_res)

print("Extracted image metadata fields: main_image_url, num_images, hi_res_images")


In [ ]:
df['timestamp_utc'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)

In [ ]:
df.head()

In [ ]:
def safe_parse_categories(x):
    try:
        return json.loads(x) if isinstance(x, str) and x.strip().startswith('[') else []
    except:
        return []
df['parsed_categories'] = df['categories'].apply(safe_parse_categories)

df['category_depth'] = df['parsed_categories'].apply(len)

# Value counts of category depth
print("Category Depth Distribution:")
print(df['category_depth'].value_counts().sort_index())

df['category_depth'].value_counts().sort_index().plot(kind='bar', title='Category Depth Distribution')
plt.xlabel("Number of Levels")
plt.ylabel("Number of Products")
plt.show()

### From above plot, to extract up to 5 levels (cat_0 to cat_4 for column 'categories')

In [ ]:
df['verified_purchase_flag'] = df['verified_purchase'].apply(lambda x: 1 if x is True else 0)

In [ ]:
def parse_category_list(val):
    if isinstance(val, list):
        return val
    elif isinstance(val, str) and val.strip().startswith("["):
        try:
            parsed = ast.literal_eval(val)
            return parsed if isinstance(parsed, list) else []
        except:
            return []
    return []

# Parse and extract category levels
df['parsed_categories'] = df['categories'].apply(parse_category_list)

for i in range(5):
    df[f'cat_{i}'] = df['parsed_categories'].apply(
        lambda lst: lst[i].strip().lower() if i < len(lst) else np.nan
    )

# Drop parsed_categories to clean up
df.drop(columns=['parsed_categories'], inplace=True)

print("Extracted cat_0 to cat_4 and dropped 'parsed_categories'.")

In [ ]:
# Columns to drop
columns_to_drop = ['verified_purchase', 'main_category', 'categories', 'asin', 'features', 'description', 'images_meta', 'timestamp']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

print(f"Dropped columns: {columns_to_drop}")

# Log transform price column
df['price_log'] = np.log1p(df['price'])

# Create new feature BPR implementation (Rationale: Plot below)
df['helpful_vote_clipped'] = df['helpful_vote'].clip(upper=5)

# Feature engineer a new average_rating column by parent
df['avg_rating_parent'] = df.groupby('parent_asin')['rating'].transform('mean').round(1)

In [ ]:
df.head()

In [ ]:
# Drop all rows where cat_0 is nan
df = df[~df['cat_0'].isna()].reset_index(drop=True)

# Perform imputation for cat_4 (fallback to cat_3 if NaN)

# Fill NaN values in cat_4 with values from cat_3
df['cat_4'] = df['cat_4'].fillna(df['cat_3'])

before = len(df)

# Drop duplicate rows
df = df.drop_duplicates().reset_index(drop=True)

after = len(df)
removed = before - after

print(f"Removed {removed:,} duplicate rows. Final row count: {after:,}")

In [ ]:
for col in ['cat_0', 'cat_1', 'cat_2', 'cat_3', 'cat_4']:
    if col in df.columns:
        print(f"\nTop categories in {col}:")
        print(df[col].value_counts(dropna=False).head(10))

## From above, confirmed that the above cat_0 can be considered as 'sports & outdoors'

In [ ]:
# Therefore its safe to drop cat_0 now

df.drop(columns=['cat_0'], inplace=True)

In [ ]:
df.head()

## Analysis of Final Table

In [ ]:
print("Data types in DataFrame:")
print(df.dtypes)

In [ ]:
print("\nDistribution of helpful_vote:")
print(df['helpful_vote'].describe())

print("\nDistribution of rating:")
print(df['rating'].value_counts().sort_index())

print("\nPrice Statistics:")
print(df['price'].describe())

min_price = df['price'].min()
max_price = df['price'].max()
print(f"\nPrice Range: ${min_price:.2f} to ${max_price:.2f}")

print("\nLog-Transformed Price Statistics:")
print(df['price_log'].describe())

In [ ]:
for col in ['cat_1', 'cat_2', 'cat_3', 'cat_4']:
    if col in df.columns:
        print(f"\nTop categories in {col}:")
        print(df[col].value_counts(dropna=False).head(10))

In [ ]:
# Given there are NaN in cat_3 and cat_4, will perform backfill referencing from cat_2

df['cat_3'] = df['cat_3'].fillna(df['cat_2'])
df['cat_4'] = df['cat_4'].fillna(df['cat_3'])  # This now fills from the new cat_3

In [ ]:
for col in ['cat_1', 'cat_2', 'cat_3', 'cat_4']:
    if col in df.columns:
        print(f"\nTop categories in {col}:")
        print(df[col].value_counts(dropna=False).head(10))

In [ ]:
# Create 1 row with 4 subplots
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Top categories in cat_1
df['cat_1'].value_counts(dropna=False).head(10).plot(
    kind='barh', ax=axes[0], color='darkorange'
)
axes[0].set_title("Top Categories - cat_1")
axes[0].invert_yaxis()

# Top categories in cat_2
df['cat_2'].value_counts(dropna=False).head(10).plot(
    kind='barh', ax=axes[1], color='seagreen'
)
axes[1].set_title("Top Categories - cat_2")
axes[1].invert_yaxis()

# Top categories in cat_3
df['cat_3'].value_counts(dropna=False).head(10).plot(
    kind='barh', ax=axes[2], color='skyblue'
)
axes[2].set_title("Top Categories - cat_3")
axes[2].invert_yaxis()

# Top categories in cat_4
df['cat_4'].value_counts(dropna=False).head(10).plot(
    kind='barh', ax=axes[3], color='orchid'
)
axes[3].set_title("Top Categories - cat_4")
axes[3].invert_yaxis()

plt.tight_layout()
plt.show()

### Based on the above plot, cold start basket collection will be:
1. camping & hiking
2. cycling
3. fitness technology
4. strength training equipment
5. cardio training

In [ ]:
def plot_top10_avg_rating_by_category(cat_col, color, ax):
    top_10 = df[cat_col].value_counts().head(10).index
    avg_rating = (
        df[df[cat_col].isin(top_10)]
        .groupby(cat_col)['rating']
        .mean()
        .sort_values(ascending=False)
    )
    
    avg_rating.plot(kind='bar', color=color, ax=ax)
    ax.set_title(f"Avg Rating by Top 10 {cat_col} Categories")
    ax.set_ylabel("Average Rating")
    ax.set_xlabel(cat_col)
    ax.set_xticklabels(avg_rating.index, rotation=45, ha='right')

    for i, v in enumerate(avg_rating.values):
        ax.text(i, v + 0.03, f"{v:.2f}", ha='center', fontsize=8)

# Create subplots in a single horizontal row
fig, axes = plt.subplots(1, 4, figsize=(28, 10))  # Wider width to fit 4 plots

# Plot each category level
plot_top10_avg_rating_by_category('cat_1', 'salmon', axes[0])
plot_top10_avg_rating_by_category('cat_2', 'mediumseagreen', axes[1])
plot_top10_avg_rating_by_category('cat_3', 'skyblue', axes[2])
plot_top10_avg_rating_by_category('cat_4', 'orchid', axes[3])

plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Clipped at 5
df['helpful_vote'].clip(upper=5).hist(bins=6, ax=axes[0])
axes[0].set_title("Helpful Votes (Capped at 5)")
axes[0].set_xlabel("helpful_vote")
axes[0].set_ylabel("Frequency")
axes[0].grid(True)

# Plot 2: Unclipped
df['helpful_vote'].hist(bins=100, ax=axes[1])
axes[1].set_title("Helpful Votes (Full Range)")
axes[1].set_xlabel("helpful_vote")
axes[1].set_ylabel("Frequency")
axes[1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Rating distribution
df['rating'].value_counts().sort_index().plot(kind='bar', ax=axes[0])
axes[0].set_title("Rating Distribution")
axes[0].set_xlabel("Rating")
axes[0].set_ylabel("Count")
axes[0].grid(True)

# Plot 2: Price (clipped at $200)
df['price'].clip(upper=200).hist(bins=50, ax=axes[1])
axes[1].set_title("Price Distribution (Capped at $200)")
axes[1].set_xlabel("Price ($)")
axes[1].set_ylabel("Frequency")
axes[1].grid(True)

# Plot 3: Price (full range)
df['price'].hist(bins=100, ax=axes[2])
axes[2].set_title("Price Distribution (Full Range)")
axes[2].set_xlabel("Price ($)")
axes[2].set_ylabel("Frequency")
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
df.head()

In [ ]:
# df to be filtered with only rows containing the top 5 categories
top_cat2 = [
    "camping & hiking",
    "cycling",
    "fitness technology",
    "strength training equipment",
    "cardio training"
]

# Filter the DataFrame
top_5_df = df[df['cat_2'].isin(top_cat2)].copy()

# Output the result
print(f"Filtered top_5_df created with {top_5_df.shape[0]:,} rows and {top_5_df.shape[1]} columns.")


In [ ]:
top_5_df.head()

In [ ]:
# Total number of rows
total_rows = len(top_5_df)
print(f"Total rows in top_5_df: {total_rows:,}")

# Total number of rows
total_rows = len(df)
print(f"Total rows in df: {total_rows:,}")

# Export 2 files
- Final_Table (labelled as: df) - total of 1,487,892 (backup in case phase 2 team needs more than 5 categories)
- 5_Category_Final_Table (labelled as: top_5_df) - 767,684

### Please note the nan values existing across these columns - to decide if drop rows, imputation etc. to proceed with model development:

In [ ]:
# Compute missing values for df
df_na = df.isna().sum().reset_index()
df_na.columns = ['Column', 'NaN in df']
df_na = df_na[df_na['NaN in df'] > 0]  # optional: show only columns with NaNs

# Compute missing values for top_5_df
top_5_na = top_5_df.isna().sum().reset_index()
top_5_na.columns = ['Column', 'NaN in top_5_df']
top_5_na = top_5_na[top_5_na['NaN in top_5_df'] > 0]  # optional: show only columns with NaNs

# Display both
print("Missing Values in df:")
display(df_na)

print("\nMissing Values in top_5_df:")
display(top_5_na)

In [ ]:
output_path = "data"

In [ ]:
# Export top 5 category subset dataset
top_5_df.to_parquet(f"{output_path}/5_category_final_table_(767,684 rows).parquet", index=False)

# Export full cleaned dataset - backup for phase 2 team in case you all need more categories
df.to_parquet(f"{output_path}/final_table_(1,487,892 rows).parquet", index=False)

print("Export completed: final_table.parquet and 5_category_final_table.parquet saved")

In [ ]:
new_1 = pd.read_parquet('data/5_category_final_table_(767,684 rows).parquet')
new_1.head()

In [ ]:
new_2 = pd.read_parquet('data/final_table_(1,487,892 rows).parquet')
new_2.head()